In [10]:
import pandas as pd
import random
import numpy as np

complete=pd.read_csv('our_data.csv')
results=pd.read_csv('RegularSeasonCompactResults.csv')

In [11]:
complete.columns

Index([u'Team_Id', u'Team_Name', u'Average Salary', u'2016 ESPN BPI',
       u'2015 ESPN BPI', u'2014 ESPN BPI', u'2013 ESPN BPI', u'2012 ESPN BPI',
       u'Moore 2016', u'Moore 2015', u'Moore 2014', u'Moore 2013',
       u'Moore 2012', u'Moore 2011', u'Kenpom 2016', u'Kenpom 2015',
       u'Kenpom 2014', u'Kenpom 2013', u'Kenpom 2012'],
      dtype='object')

In [12]:
results.columns

Index([u'Season', u'Daynum', u'Wteam', u'Wscore', u'Lteam', u'Lscore', u'Wloc',
       u'Numot'],
      dtype='object')

In [137]:
#Baseline1 . For match between A and B, predict one who had highest wins in the matches among these two.
#Interestingly there were mostly
from sklearn.metrics import f1_score
def get_baseline(year, results):
    results=results[results['Season']==year]
    f_results=results[['Wteam', 'Lteam', 'Wscore', 'Lscore']]
    y_predict=[]
    y_actual=[]
    for index,row in f_results.iterrows():
        team1=row['Wteam']
        team2=row['Lteam']
        team_results=results.loc[((results['Wteam']==team1) | (results['Lteam']==team1))
                           & ((results['Wteam']==team2) | (results['Lteam']==team2))]
        totalmatches=team_results.shape[0]
        teamA_matches_won=team_results[team_results['Wteam']==row['Wteam']]
        teamA_matches_won=teamA_matches_won.shape[0]
        teamB_matches_won=totalmatches-teamA_matches_won
        if teamA_matches_won>teamB_matches_won:
            y_predict.append(1)
        else:
            y_predict.append(-1)
        y_actual.append(1)
    return f1_score(y_actual,y_predict)
        
    

In [138]:
get_baseline(2015,results)

0.89380165289256197

In [131]:
#Baseline2 . For match between A and B, predict one who had highest wins in the history.
from sklearn.metrics import f1_score
def get_baseline2(year, results):
    results=results[results['Season']==year]
    f_results=results[['Wteam', 'Lteam', 'Wscore', 'Lscore']]
    y_predict=[]
    y_actual=[]
    for index,row in f_results.iterrows():
        team1=row['Wteam']
        team2=row['Lteam']
        teamA_matches_won=results.loc[(results['Wteam']==team1)].shape[0]
        teamB_matches_won=results.loc[(results['Wteam']==team2)].shape[0]
        #print teamA_matches_won,' ',teamB_matches_won
        if teamA_matches_won>teamB_matches_won:
            y_predict.append(1)
        else:
            y_predict.append(-1)
        y_actual.append(1)
    return f1_score(y_actual,y_predict)
        
    

In [132]:
get_baseline2(2015,results)

0.83608695652173914

In [13]:
def get_data(year,results,data):
    results=results[results['Season']==year]
    cols=data.columns
    selected_cols=[]
    selected_cols.append('Average Salary')
    for col in cols:
        if str(year) in col:
            selected_cols.append(col)
    X=[]
    y=[]
    xxx=0
    for index,row in results.iterrows():
        xrow=[]
        toss=random.randint(0,234234988)
        if toss%2==0:
            result=row['Wscore']-row['Lscore']
            y.append(result)
            team1=data[data['Team_Id']==row['Wteam']]
            for col in selected_cols:
                val=team1.iloc[0][col]
                if np.isnan(val):
                    val=data[col].min()
                xrow.append(val)
            team2=data[data['Team_Id']==row['Lteam']]
            for col in selected_cols:
                val=team2.iloc[0][col]
                if np.isnan(val):
                    val=data[col].min()
                xrow.append(val)
            
        else:
            result=row['Lscore']-row['Wscore']
            y.append(result)
            team2=data[data['Team_Id']==row['Lteam']]
            for col in selected_cols:
                val=team2.iloc[0][col]
                if np.isnan(val):
                    val=data[col].min()
                xrow.append(val)
            team1=data[data['Team_Id']==row['Wteam']]
            for col in selected_cols:
                val=team1.iloc[0][col]
                if np.isnan(val):
                    val=data[col].min()
                xrow.append(val)
        X.append(xrow)

        xxx=xxx+1

    return [np.array(X),np.array(y)];
    
        
        

In [77]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split

d=get_data(2015,results,complete)
newy=[]
for n in d[1]:
    if n>0:
        newy.append(1)
    elif n<0:
        newy.append(-1)
    else:
        newy.append(0)
        
X_train, X_test, y_train, y_test = train_test_split(d[0], newy, test_size=0.20)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.25)

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print 'Gaussian Naive Bayes f1score ',f1_score(y_test, y_pred)



Gaussian Naive Bayes f1score  0.741108354012


In [145]:
from sklearn.svm import SVR
from sklearn import preprocessing

d=get_data(2015,results,complete)

d[0]=preprocessing.normalize(d[0])      

X_train, X_test, y_train, y_test = train_test_split(d[0], d[1], test_size=0.20)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.25)

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf = svr_rbf.fit(X_train, y_train);
y_rbf = svr_rbf.predict(X_test)

y_actual_discrete=[]
for n in y_test:
    if n>0:
        y_actual_discrete.append(1)
    elif n<0:
        y_actual_discrete.append(-1)
    else:
        y_actual_discrete.append(0)
        
y_predict_discrete=[]
for n in y_rbf:
    if n>0:
        y_predict_discrete.append(1)
    elif n<0:
        y_predict_discrete.append(-1)
    else:
        y_predict_discrete.append(0)
        
print 'Test F1 score :',f1_score(y_actual_discrete,y_predict_discrete)

y_rbf = svr_rbf.predict(X_train)

y_actual_discrete=[]
for n in y_train:
    if n>0:
        y_actual_discrete.append(1)
    elif n<0:
        y_actual_discrete.append(-1)
    else:
        y_actual_discrete.append(0)
        
y_predict_discrete=[]
for n in y_rbf:
    if n>0:
        y_predict_discrete.append(1)
    elif n<0:
        y_predict_discrete.append(-1)
    else:
        y_predict_discrete.append(0)
        
print 'Train F1 score :',f1_score(y_actual_discrete,y_predict_discrete)

Test F1 score : 0.526960784314
Train F1 score : 0.504734458625


In [ ]:
import csv

def writeListofListToCsvFile(fname, data):
    """
    @param fname: string, name of file to write
    @param data: list of list of items

    Write data to file
    """
    mycsv = csv.writer(open(fname, 'wb'))
    for row in data:
        mycsv.writerow(row)

In [14]:
#grid search for C and lambda
from sklearn.grid_search import ParameterGrid
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVR

d=get_data(2015,results,complete)

d[0]=preprocessing.normalize(d[0])      

X_train, X_test, y_train, y_test = train_test_split(d[0], d[1], test_size=0.20)

param_grid = {'C':[2**-5,2**-3,2**-1,2**1,2**3,2**5,2**7,2**9,2**11,2**13,2**15,2**17,2**19],
              'gamma': [2**-15,2**-13,2**-12,2**-11,2**-9,2**-7,2**-5,2**-3,2**-2,2**1,2**3,2**5]}
results=[]
l=len(ParameterGrid(param_grid))
i=1
for obj in ParameterGrid(param_grid):
    if i%(l/10)==0:
        print 'Iteration ',i
    i=i+1
    result=[]
    result.append(obj['C'])
    result.append(obj['gamma'])
    svr_rbf = SVR(kernel='rbf', C=obj['C'], gamma=obj['gamma'])
    svr_rbf = svr_rbf.fit(X_train, y_train);
    y_rbf = svr_rbf.predict(X_test)

    y_actual_discrete=[]
    for n in y_test:
        if n>0:
            y_actual_discrete.append(1)
        elif n<0:
            y_actual_discrete.append(-1)
        else:
            y_actual_discrete.append(0)
            
    y_predict_discrete=[]
    for n in y_rbf:
        if n>0:
            y_predict_discrete.append(1)
        elif n<0:
            y_predict_discrete.append(-1)
        else:
            y_predict_discrete.append(0)
        
    result.append(f1_score(y_actual_discrete,y_predict_discrete))

    y_rbf = svr_rbf.predict(X_train)

    y_actual_discrete=[]
    for n in y_train:
        if n>0:
            y_actual_discrete.append(1)
        elif n<0:
            y_actual_discrete.append(-1)
        else:
            y_actual_discrete.append(0)
        
    y_predict_discrete=[]
    for n in y_rbf:
        if n>0:
            y_predict_discrete.append(1)
        elif n<0:
            y_predict_discrete.append(-1)
        else:
            y_predict_discrete.append(0)
        
    result.append(f1_score(y_actual_discrete,y_predict_discrete))
    results.append(result)
writeListofListToCsvFile('svm_rbf.csv',results)

Iteration  18
Iteration  36
Iteration  54
Iteration  72
Iteration  90
Iteration  108
Iteration  126
Iteration  144
Iteration  162
Iteration  180


/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


NameError: name 'writeListofListToCsvFile' is not defined

In [16]:
writeListofListToCsvFile('svm_rbf.csv',results)